In [1]:
import math
import warnings
%matplotlib inline
warnings.filterwarnings("ignore")
from qiskit import QuantumCircuit
from qiskit.circuit.library import GroverOperator, MCMT, ZGate
from qiskit.visualization import plot_distribution
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
import random
import string

def grover_oracle(marked_states):
    """Build a Grover oracle for multiple marked states

    Here we assume all input marked states have the same number of bits

    Parameters:
        marked_states (str or list): Marked states of oracle

    Returns:
        QuantumCircuit: Quantum circuit representing Grover oracle
    """
    if not isinstance(marked_states, list):
        marked_states = [marked_states]

    num_qubits = len(marked_states[0])

    qc = QuantumCircuit(num_qubits)

    for target in marked_states:
        rev_target = target[::-1]
        zero_inds = [ind for ind in range(num_qubits) if rev_target.startswith("0", ind)]
        qc.x(zero_inds)
        qc.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)
        qc.x(zero_inds)
        
    return qc

In [169]:
service = QiskitRuntimeService(channel="ibm_quantum", token="f14aedea4186539cd7cf897268a931ade27b17266541c58fc77f3115dea2cb4b0559710e085ebc9a80098f0abb408a716dd8c15ee2c8ca9b11d9b66d80c62e2b")
backend = service.least_busy(operational=True, simulator=False)
backend.name

'ibm_kyoto'

In [173]:

random_numbers = [(bin(x)[2:]).zfill(3) for x in list(range(0, 8))]


# if 12 not in states:
#     binary_representation = bin(12)
#     el_to_search = binary_representation
#     states.append(binary_representation)

random.shuffle(random_numbers)

for x in random_numbers:
    print(f"{x}")

random_el_to_search = random_numbers[random.randint(0, 8)]


100
010
110
101
011
000
111
001


In [174]:

marked_states = random_el_to_search
oracle = grover_oracle(marked_states)

grover_op = GroverOperator(oracle)

optimal_num_iterations = math.floor(
    math.pi / (4 * math.asin(math.sqrt(len(marked_states) / 2**grover_op.num_qubits)))
)
qc = QuantumCircuit(grover_op.num_qubits)
qc.h(range(grover_op.num_qubits))

qc.compose(grover_op.power(optimal_num_iterations), inplace=True)

qc.measure_all()

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

circuit_isa = pm.run(qc)

sampler = Sampler(backend=backend)
sampler.options.default_shots = 10_000
result = sampler.run([circuit_isa]).result()
dist = result[0].data.meas.get_counts()

In [178]:
dist
# random_el_to_search

{'011': 1769,
 '000': 4682,
 '001': 556,
 '101': 817,
 '100': 446,
 '111': 603,
 '110': 474,
 '010': 653}